In [2]:
import scripts.sql_parser.sqlite_parser as sql
from scripts.sql_parser.constants import *
from scripts.sql_parser.table_operations import get_table

NC_PATH = '../../data_sets/db/northwind.db'
SC_PATH = '../../data_sets/db/sakila.db'
CC_PATH = '../../data_sets/db/covid19.db'

# instantiate parser objects based on the db path
nc, sc, cc = sql.SqlParser(NC_PATH), sql.SqlParser(SC_PATH), sql.SqlParser(CC_PATH)

# create a table reference for each db for faster querying
nct, sct, cct = nc.table, sc.table, cc.table

In [3]:
# Get all Northwind tables
nc.q(sql.ALL_TABLES)

,name
0,Categories
1,sqlite_sequence
2,CustomerCustomerDemo
3,CustomerDemographics
4,Customers
5,Employees
6,EmployeeTerritories
7,Order Details
8,Orders
9,Products


In [4]:
# handle Aggregation functions

nct(name='Order Details',
    cols=['avg=UnitPrice:UP Avg',
          'max=UnitPrice:UP Max',
          'min=UnitPrice:UP Min',
          'count=UnitPrice:UP Count'])


SELECT 
	AVG(UnitPrice) AS "UP Avg",
	MAX(UnitPrice) AS "UP Max",
	MIN(UnitPrice) AS "UP Min",
	COUNT(UnitPrice) AS "UP Count"
FROM "Order Details" 


,UP Avg,UP Max,UP Min,UP Count
0,28.850379,263.5,2,609283


In [5]:
# supports most of the clauses (except GROUP BY)

nct(name='Order Details',
    cols=['UnitPrice'], distinct=True,
    where=['&20.0 >= UnitPrice <= 70.0', '|UnitPrice > 20'],
    order_by=('UnitPrice', -1),
    limit=5, offset=2)


SELECT DISTINCT 
	UnitPrice
FROM "Order Details" 
WHERE UnitPrice BETWEEN 20.0 and 70.0
	OR UnitPrice > 20
ORDER BY UnitPrice DESC
LIMIT 5 OFFSET 2


,UnitPrice
0,123.79
1,99.00
2,97.00
3,81.00
4,77.60


In [6]:
# simple join (between 2 tables)

nc.join(table_left={NAME: 'Customers', COLS: ['CompanyName', 'Phone', 'Fax']},
        table_right={NAME: 'Orders:o', COLS: ['ShipRegion', 'ShipCountry']},
        shared_col='CustomerID',
        join=INNER,
        limit=5)


SELECT 
	Customers.CompanyName,
	Customers.Phone,
	Customers.Fax,
	o.ShipRegion,
	o.ShipCountry
FROM Customers
INNER JOIN Orders o
ON Customers.CustomerID = o.CustomerID
LIMIT 5


,CompanyName,Phone,Fax,ShipRegion,ShipCountry
0,Vins et alcools Chevalier,26.47.15.10,26.47.15.11,Western Europe,France
1,Toms Spezialitäten,0251-031259,0251-035695,Western Europe,Germany
2,Hanari Carnes,(21) 555-0091,(21) 555-8765,South America,Brazil
3,Victuailles en stock,78.32.54.86,78.32.54.87,Western Europe,France
4,Suprêmes délices,(071) 23 67 22 20,(071) 23 67 22 21,Western Europe,Belgium


In [7]:
# multi join (between more than 2 tables, based on different join types)

customers_table = get_table(name='Customers:C',
                            shared='CustomerID',
                            cols=['CompanyName:MyCompany', 'Phone:                 MyPhone', 'Fax:MyFax'])
orders_table = get_table(name='Orders:O',
                         shared='CustomerID',
                         cols=['ShipRegion', 'ShipCountry'],
                         join=INNER)
order_details_table = get_table(name='Order Details:OD',
                                shared='OrderId',
                                cols=['ProductId', 'Quantity', 'UnitPrice'],
                                join=LEFT)

nc.multi_join(tables=([customers_table, orders_table, order_details_table]),
              starts_with=('Phone', '3'),
              order_by=('ShipRegion', 1),
              limit=5,
              distinct=True)


SELECT DISTINCT 
	C.CompanyName AS MyCompany,
	C.Phone AS MyPhone,
	C.Fax AS MyFax,
	O.ShipRegion,
	O.ShipCountry,
	OD.ProductId,
	OD.Quantity,
	OD.UnitPrice
FROM Customers C
INNER JOIN Orders O ON C.CustomerID = O.CustomerID
LEFT JOIN "Order Details" OD ON O.CustomerID = OD.OrderId
WHERE Phone LIKE '3%'
ORDER BY ShipRegion ASC
LIMIT 5


,MyCompany,MyPhone,MyFax,ShipRegion,ShipCountry,ProductID,Quantity,UnitPrice
0,Simons bistro,31 12 34 56,31 13 35 57,British Isles,UK,None,None,None
1,La corne d'abondance,30.59.84.10,30.59.85.11,British Isles,UK,None,None,None
2,La corne d'abondance,30.59.84.10,30.59.85.11,British Isles,Ireland,None,None,None
3,La corne d'abondance,30.59.84.10,30.59.85.11,Central America,Mexico,None,None,None
4,Simons bistro,31 12 34 56,31 13 35 57,Central America,Mexico,None,None,None


In [8]:
# multi join using dicts

nc.multi_join(
    tables=([{NAME: 'Customers:C', SHARED: 'CustomerID', COLS: ['CompanyName:Company Name', 'Phone:Land Phone']},
             {NAME: 'Orders:O', SHARED: 'CustomerID', COLS: ['ShipCountry'], JOIN: INNER},
             {NAME: 'Order Details:OD', SHARED: 'OrderId', COLS: ['ProductId', 'Quantity', 'UnitPrice: Price'],
              JOIN: LEFT}]),
    limit=5)


SELECT 
	C.CompanyName AS "Company Name",
	C.Phone AS "Land Phone",
	O.ShipCountry,
	OD.ProductId,
	OD.Quantity,
	OD.UnitPrice AS Price
FROM Customers C
INNER JOIN Orders O ON C.CustomerID = O.CustomerID
LEFT JOIN "Order Details" OD ON O.CustomerID = OD.OrderId
LIMIT 5


,Company Name,Land Phone,ShipCountry,ProductID,Quantity,Price
0,Vins et alcools Chevalier,26.47.15.10,France,None,None,None
1,Toms Spezialitäten,0251-031259,Germany,None,None,None
2,Hanari Carnes,(21) 555-0091,Brazil,None,None,None
3,Victuailles en stock,78.32.54.86,France,None,None,None
4,Suprêmes délices,(071) 23 67 22 20,Belgium,None,None,None


In [9]:
nct(name='Orders',
    cols=['ShipCountry', 'count=*:OrderCount'],
    group_by='ShipCountry',
    order_by=('OrderCount', -1),
    limit=-5)


SELECT 
	ShipCountry,
	COUNT("*") AS OrderCount
FROM Orders 
GROUP BY ShipCountry
ORDER BY OrderCount DESC
LIMIT 5


,ShipCountry,OrderCount
0,USA,2328
1,Germany,2193
2,France,1778
3,Brazil,1683
4,UK,1280


In [13]:
 # find the total sales amount for each category and supplier
nc.multi_join(
    tables=[{NAME: 'Categories', SHARED: 'CategoryId', COLS: ['CategoryName']},
            {NAME: 'Products', SHARED: 'CategoryId', JOIN: INNER},
            {NAME: 'Suppliers', SHARED: 'SupplierId', COLS: ['CompanyName'], JOIN: INNER},
            {NAME: 'Order Details', SHARED: 'ProductId', COLS: [], JOIN: INNER}],

    col_agg={AGG_FUNC: 'sum:TotalSales',
             COLS: ['Products.UnitPrice', 'Order Details.Quantity'],
             COLS_OP: '*'},

    group_by=['Categories.CategoryName', 'Suppliers.CompanyName']
)


SELECT 
	Categories.CategoryName,
	Suppliers.CompanyName,
	SUM(Products.UnitPrice * "Order Details".Quantity) AS TotalSales
FROM Categories
INNER JOIN Products ON Categories.CategoryId = Products.CategoryId
INNER JOIN Suppliers ON Products.CategoryId = Suppliers.SupplierId
INNER JOIN "Order Details" ON Suppliers.SupplierId = "Order Details".ProductId
Group By Categories.CategoryName, Suppliers.CompanyName


,CategoryName,CompanyName,TotalSales
0,Beverages,Exotic Liquids,92018203.75
1,Condiments,New Orleans Cajun Delights,55848703.50
2,Confections,Grandma Kelly's Homestead,66130996.88
3,Dairy Products,Tokyo Traders,57093979.80
4,Grains/Cereals,Cooperativa de Quesos 'Las Cabras',28297127.25
5,Meat/Poultry,Mayumi's,65298920.60
6,Produce,"Pavlova, Ltd.",33012544.50
7,Seafood,"Specialty Biscuits, Ltd.",50108071.86


In [11]:
cct(name='IndiaCases',
    cols=['Confirmed', 'Date'],
    order_by=['confirmed', -1],
    limit=5)


SELECT 
	Confirmed,
	Date
FROM IndiaCases 
ORDER BY confirmed DESC
LIMIT 5


,Confirmed,Date
0,94041,2020-06-11
1,90787,2020-06-10
2,88528,2020-06-09
3,85975,2020-06-08
4,82968,2020-06-07
